<h1 dir="rtl">مقدمه‌ای درباره اصول نقشه نگاری</h1>

<br />

<a href="https://www.youtube.com/watch?v=22NO4ReFCzY&list=PLY_JJbHwd6kftjlcbtChIKZfEPEfFRGcY&index=20&ab_channel=TheInternationalJournalists%27Network">
<img src="https://dotnet-talk.com/images/icfj/2_3.jpg" width="50%" alt="اصول نقشه نگاری - ویدیو" />
</a>

<br />

<p dir="rtl">می‌توانیم با استفاده از چندین کتابخانه Python به راحتی مجموعه‌داده‌ها را نقشه نگاری کنیم. یکی از ساده‌ترین کتابخانه‌هایی که می‌توانیم استفاده کنیم، به نام <span style="color: orange">folium</span> است که برای نمایش اولین mapها از آن استفاده خواهیم کرد. برای کسب اطلاعات بیشتر درباره folium <a href="https://python-visualization.github.io/folium/">اینجا</a> کلیک کنید. همچنین در پایان این درس، منابع جایگزین را نیز ارائه خواهیم داد.</p>

In [ ]:
# First, we have to install both folium and geopandas packages. 

!pip3 install folium==0.14.0
!pip3 install geopandas==0.13.2

In [ ]:
# importing necessary libraries

# for any data cleaning/sorting/filtering
import numpy as np
import pandas as pd

# for mapping
import folium

<h3 dir="rtl">وارد کردن و پاک‌سازی داده برای مپ کردن</h3>

<p dir="rtl">برای قسمت اول این درس، مجموعه‌داده‌ای متفاوت از دو درس قبلی داده‌بینی را مورد بررسی قرار خواهیم داد. ما قصد داریم جمعیت زندانیان در سراسر جهان در سال 2022 را از <a href="https://iranopendata.org/en/dataset/iod-03646-prison-populations-2022/resource/4c66de54-7d7f-4790-8f3c-8ee8b7857886">World Prison Brief</a> نقشه‌برداری کنیم. برای دانلود مستقیم این داده، می‌توانید <a href="https://media.iranopendata.org/IODBU2/2023/03/03/iod-03647-prison-populations-2022-en.csv">به این لینک</a> کلیک کنید. در غیر اینصورت، ما CSV را از <a href="https://raw.githubusercontent.com/ICFJ-Computational-Journaism/datasets/main/csv/iod-03647-prison-populations-2022-en.csv">این لینک github</a> وارد خواهیم کرد.</p>

In [ ]:
# We'll first need to download the data from Iran Open Data and import the csv as a dataframe, as we have been in previous lessons

prison_pop = pd.read_csv("../datasets/csv/iod-03647-prison-populations-2022-en.csv")
prison_pop # to check

<div dir="rtl">معمولاً مجموعه‌داده‌ای که می‌خواهید نقشه‌برداری کنید، از ابتدا تمیز نخواهد بود. پس از وارد کردن آن به یک dataframe، ما باید داده ها را قبل از نقشه نگاری، تمیز کنیم.</div>

<div dir="rtl">ما می‌خواهیم مطمئن شویم که مجموع جمعیت زندانیان به صورت اعداد اعشاری یا اعداد صحیح، نه مقادیر رشته‌ای باشد. و اگر این مقادیر رشته‌ای باشند، باید آنها را تبدیل کنیم! این موضوع برای مپ کردن داده‌ها در زمان بعدی بسیار مهم است.</div>


In [ ]:
# Prison population values are actually string values, not integers or float values,
# so let's change them so they can be recognized as actual numerical values

prison_pop['Prison Population Total'] = prison_pop['Prison Population Total'].astype(int)

# To check
prison_pop.dtypes

# If an error pops up, this is the right response!

<div dir="rtl">اوه! برخی از مقادیر را باید با مقادیر صحیح جایگزین کنیم (به عنوان مثال 68056 به جای 'c.68056' برای عربستان سعودی که احتمالاً یک تایپو است) تا بتوانیم مجموع جمعیت زندانیان را به طور کامل به اعداد تبدیل کنیم.
</div>
<br />
<div dir="rtl">
بیایید سطرهایی که مقادیر نادرست/نوشتاری را دارند - در این مورد، مقادیری که شامل حروف یا کاراکترهای دیگر نیز هستند - پیدا کنیم:</div>

In [ ]:
prison_pop.loc[prison_pop['Prison Population Total'].str.contains('c')]
# should output incorrect population values for Saudi Arabia and Djibouti

<div dir="rtl">حالا بیایید این مقادیر نادرست را جایگزین کنیم!</div>

In [ ]:
prison_pop = prison_pop.replace({'Prison Population Total':
                                 {'c.68056':'68056',
                                  'c.662':'662'}})

<div dir="rtl">پس از جایگزینی این مقادیر، می‌توانیم دوباره تلاش کنیم برای تبدیل داده‌ها:</div>

In [ ]:
prison_pop['Prison Population Total'] = prison_pop['Prison Population Total'].astype(int)

prison_pop # To check

<div dir="rtl">بعد از انجام عملیات پاک‌سازی، قبل از مپ کردن این داده‌ها، <strong>تنها ستون‌های لازم را انتخاب می‌کنیم</strong>. همچنین ستون‌ها را تغییر نام می‌دهیم تا درک آسان‌تری داشته باشیم و همچنین کمتر کلمات داشته و در تحلیل داده و مپ کردن آسان‌تر باشند.</div>

In [ ]:
# Selecting for only the "Title" and "Prison Population Total" columns
prison_pop_cleaned = prison_pop[["Title", "Prison Population Total"]]

# Renaming the "Title" and "Prison Population Total" columns to "name" and "population" - less wordy and more straightforward
prison_pop_cleaned = prison_pop_cleaned.rename(columns={"Title":"name", "Prison Population Total": "population"})
# prison_pop_cleaned # Uncomment to check what it looks like

<h2 dir="rtl">نقشه‌برداری</h2>

<h3 dir="rtl">ایجاد نقشه کروپلت</h3>

<div dir="rtl">این نوع مجموعه داده که نشان می‌دهد جمعیت زندانیان در هر کشور را نشان می‌دهد، می‌تواند از طریق یک <strong>نقشه کروپلت</strong> به خوبی نمایش داده شود. نقشه کروپلت نوعی نقشه است که داده‌ها را - به خصوص داده‌های مرتبط با مرزهای پیش‌تعیین شده مانند مرزهای کشور یا شهرستان‌ها - با تنظیم سایه‌بندی، رنگ یا الگو نشان می‌دهد.</div>
<br />
<div dir="rtl">نقشه‌های کروپلت بر اساس دو نوع داده وابسته هستند: نوع اول داده‌هایی مانند اعداد جمعیت زندانیان است و نوع دوم داده‌هایی است که نام‌های کشورهای متناظر با این اعداد جمعیت زندانیان را به اطلاعات مکانی و جغرافیایی مرتبط می‌کند.</div>
<br />
<div dir="rtl">هم‌اکنون باید نوع دوم، یعنی اطلاعات مکانی را وارد کنیم! سازندگان کتابخانه پایتون folium که از آن برای ایجاد نقشه استفاده می‌کنیم، یک نمونه با <a href="https://raw.githubusercontent.com/python-visualization/folium/main/examples/data/world-countries.json">پرونده JSON</a> دارند که شامل این اطلاعات مکانی برای کشورهای مختلف جهان است. (JSON یا JavaScript Object Notation فرمت رایجی برای ذخیره‌سازی داده‌های استفاده شده توسط زبان برنامه‌نویسی جاوااسکریپت است. در این حالت، داده‌های ما در قالب یک GeoJSON قرار دارد که یک نوع استاندارد JSON برای رمزگذاری داده‌های جغرافیایی است.)</div>


In [ ]:
# Importing the world countries JSON

world_countries = "../datasets/json/world-countries.json"

<div dir="rtl">هم‌اکنون بیایید نقشه choropleth واقعی را ایجاد کنیم. ابتدا باید یک نقشه خالی را شروع کنیم و آن را به یک متغیر اختصاص دهیم.</div>
<br />
<div dir="rtl">برای مرکز کردن نقشه جهان بر روی ایران، مکان را به 32.4279، 53.6880 تنظیم می‌کنیم - یعنی 32.4279 درجه شمال و 53.6880 شرق.</div>

In [ ]:
# Creating the blank map

map = folium.Map(location=[32.4279, 53.6880], zoom_start=2.5)
#map # Uncomment to check
map

<div dir="rtl">سپس داده‌های جمعیت زندان و داده‌های مکانی را به روی این نقشه خالی اضافه می‌کنیم!</div>
<br />
<div dir="rtl">بیایید پارامترهای تابع <code>folium.Choropleth()</code> را تفکیک کنیم:</div>
<br />
<ul dir="rtl">
<li dir="rtl"><code>geo_data=world_countries</code>: داده‌های مکانی از کجا به دست می‌آیند؟ در این حالت، از پرونده JSON کشورهای جهان استفاده می‌کنیم که در متغیر <code>world_countries</code> ذخیره شده است.</li>
<li dir="rtl"><code>name="choropleth"</code>: این یک نقشه از چه نوعی است؟</li>
<li dir="rtl"><code>data=prison_pop_cleaned</code>: داده نوع دیگری که می‌خواهیم از آن نمایش دهیم، از کجا به دست می‌آید؟ در این حالت، از دیتافریم/CSV جمعیت زندان استفاده می‌کنیم.</li>
<li dir="rtl"><code>columns=["name", "population"]</code>: کشورهایی که مایل به نمایش دادن اطلاعات آنها هستیم کدامند؟</li>
<li dir="rtl"><code>key_on="feature.properties.name"</code>: کدام ویژگی یا "ستون" در پرونده JSON کشورهای جهان با ستونی در داده‌های جمعیت زندان در <code>prison_pop_cleaned</code> مطابقت دارد و اطلاعات مکانی را نگه می‌دارد؟ در این حالت، باید ستونی که نام کشورها را نگه می‌دارد را انتخاب کنید.</li>
<li dir="rtl"><code>fill_color="OrRd"</code>: باید یک "کد ColorBrewer معتبر" باشد - به عبارت دیگر، رنگ پرکردن باید به یک فرمت خاص تطابق داشته باشد! می‌توانید از هر یک از گروه‌های رنگی پیوسته در <a href="">این لیست</a> استفاده کنید.</li>
<li dir="rtl"><code>fill_opacity=0.7</code>: شفافیت پرکردن شکل.</li>
<li dir="rtl"><code>line_opacity=0.1</code>: شفافیت خط هر شکل.</li>
<li dir="rtl"><code>legend_name="جمعیت زندان"</code>: عنوان منوی راهنما را چه نامی می‌خواهید داشته باشید؟</li>
<li dir="rtl"><code>highlight=True</code>: فعال کردن نمایش برجسته هنگام موس روی شکل.</li>
</ul>
<div dir="rtl">توجه: اگر این را در یک دفترچه یادداشت Jupyter محلی مشاهده می‌کنید و قادر به دیدن آن نیستید، ممکن است نیاز داشته باشید آن را به عنوان یک دفترچه یادداشت قابل اعتماد تنظیم کنید. در بالای چپ دفترچه یادداشت Jupyter می‌توانید به مسیر File -> Trust Notebook بروید.</div>

In [ ]:
# Creating the actual choropleth map by tying together these two data sources

folium.Choropleth(
    geo_data=world_countries,
    name="choropleth", 
    data=prison_pop_cleaned,
    columns=["name", "population"],
    key_on="feature.properties.name",
    fill_color="OrRd",
    fill_opacity=0.7,
    line_opacity=0.1,
    legend_name="Prison population",
    highlight=True
).add_to(map) # add this data to the blank map

folium.LayerControl().add_to(map)
map

<div dir="rtl">
ما در حال حاضر یک نقشه choropleth دارای جمعیت زندان در سراسر جهان داریم! رنگ تیره‌تر نشان دهنده جمعیت زندان بیشتر است.
</div>
<br />
<div dir="rtl">
بعضی از کشورها/مناطق به دلیل عدم تطابق نام کشورها در `prison_pop_cleaned` با نام‌های کشورها در فایل JSON جهانی (`world_countries`) خاکستری شده‌اند (به عنوان مثال، "روسیه" در `world_countries` به عنوان "جمهوری روسیه" در مجموعه داده جمعیت زندان وجود دارد). بیایید این تفاوت‌ها را تصحیح کنیم!
</div>

In [ ]:
prison_pop_cleaned = prison_pop_cleaned.replace({
    'United Kingdom: England & Wales': 'United Kingdom',
    'Russian Federation': 'Russia',
    'Republic of (South) Korea': 'South Korea',
    'Greenland (Denmark)': 'Greenland',
    'Myanmar (formerly Burma)': 'Myanmar',
    'Ireland, Republic of': 'Ireland',
    'Moldova (Republic of)': 'Moldova',
    'Cyprus (Republic of)': 'Cyprus',
    "Cote d'Ivoire": 'Ivory Coast',
    'Guinea (Republic of)': 'Guinea',
    'Democratic Republic of Congo': 'Democratic Republic of the Congo',
    'Republic of Congo': 'Republic of the Congo',
    'Tanzania': 'United Republic of Tanzania',
    'eSwatini/Swaziland': 'Swaziland'
})
# Replace 'United Kingdom: England & Wales' with 'United Kingdom' - the name in 'world-countries.json'
# Replace 'Russian Federation' with 'Russia' - the name in 'world-countries.json'
# Replace 'Republic of (South) Korea' with 'South Korea' - the name in 'world-countries.json'
# Replace 'Greenland (Denmark)' with 'Greenland' - the name in 'world-countries.json'
# Replace 'Myanmar (formerly Burma)' with 'Myanmar' - the name in 'world-countries.json'
# Replace 'Ireland, Republic of' with 'Ireland' - the name in 'world-countries.json'
# Replace 'Moldova (Republic of)' with 'Moldova' - the name in 'world-countries.json'
# Replace 'Cyprus (Republic of)' with 'Cyprus' - the name in 'world-countries.json'
# Replace 'Cote d'Ivoire' with 'Ivory Coast' 
# Replace 'Guinea (Republic of)' with 'Guinea' 
# Replace 'Democratic Republic of Congo' with 'Democratic Republic of the Congo' 
# Replace 'Republic of Congo' with 'Republic of the Congo'
# Replace 'Tanzania' with 'United Republic of Tanzania' 
# Replace 'eSwatini/Swaziland' with 'Swaziland' (even though the updated name is Eswatini - 'Swaziland' is the name in 'world-countries.json')

<div dir="rtl">
این فرآیند باعث تغییر رنگ بخشی از کشورها نشده است، اما برای بخشی از کشورهای هنوز خاکستری شده، این فرآیند (تطابق نام کشورها در دو مجموعه داده) را تکرار کنید. سپس نقشه choropleth را مجدداً برسی کنید - باید تعدادی کشور خاکستری کمتر داشته باشیم.
</div>

In [ ]:
# We need to create a new base map to display this cleaned up data

new_map = folium.Map(location=[32.4279, 53.6880], zoom_start=2.5)

folium.Choropleth(
    geo_data=world_countries,
    name="choropleth", 
    data=prison_pop_cleaned,
    columns=["name", "population"],
    key_on="feature.properties.name",
    fill_color="OrRd",
    fill_opacity=0.7,
    line_opacity=0.1,
    legend_name="Prison population",
    highlight=True
).add_to(new_map)

new_map

<div dir="rtl">
    <h3>ایجاد نقشه نقطه/نقطه</h3>
</div>
<div dir="rtl">
    <p>بیایید برای یک لحظه از نقشه‌های کرافولیمتون بازموندیم و اکنون سعی کنیم یک نقشه نقطه/نقطه با استفاده از کتابخانه Folium ایجاد کنیم! این نقشه با داده‌های مکانی خاص (مانند عرض جغرافیایی و طول جغرافیایی) کار می‌کند و داده‌های حاوی "چندضلعی" را (که اغلب مرزهای جغرافیایی خاصی از شهرستان‌ها یا کشورها را مشخص می‌کنند) نمی‌پذیرد.</p>
</div>
<div dir="rtl">
    <p>ما همچنین می‌توانیم از داده‌های مختلف و غیرمربوط به زندان برای ایجاد این نقشه استفاده کنیم. بیایید یک نقشه نقطه/نقطه خود بسازیم که مکان سفارتخانه‌ها و کنسولگری‌های خارجی ایران در سراسر جهان را نشان می‌دهد. ما این داده‌های مکانی را از بخش "سفارتخانه‌ها و نمایندگی‌ها" در پورتال داده‌های باز ایران (که توسط دانشگاه سیراکیوز برپا شده است) دریافت می‌کنیم.</p>
    <p>لطفاً به <a href="https://irandataportal.syr.edu/embassies-and-missions">اینجا</a> بروید و داده‌های لازم را دانلود کنید.</p>
</div>

In [ ]:
# Let's import the GeoJSON containing the point data of Iranian embassy locations as a dataframe

embassies = pd.read_csv("../datasets/csv/iranian-embassies.csv")

embassies.head() #to check

<div dir="rtl">
<p>
حالا بیایید همان کاری که در هنگام ساخت نقشه کروپلت انجام دادیم را تکرار کنیم و یک نقشه خالی را مقداردهی و ایجاد کنیم. پس از انجام این کار، می‌توانیم نقاط را روی این نقشه خالی نمایش دهیم.
</p>
</div>

In [ ]:
# Creating a new map
point_map = folium.Map(location=[32.4279, 53.6880], zoom_start=2)

<div dir="rtl">
<p>
سپس می‌توانیم یک لایه جدید اضافه کنیم که شامل نقاط باشد (هر نقطه نمایانگر موقعیت یک سفارتخانه ایران در سراسر جهان است).
</p>
</div>

In [ ]:
# Adding markers one by one on the map
for i in range(0, len(embassies)): # We loop through the embassies dataset
  folium.Marker( # and for each embassy
      location=[embassies.iloc[i]['latitude'], embassies.iloc[i]['longitude']], # we plot the location using the latitude and longitude data points
      popup=embassies.iloc[i]['city'], # and we style the tooltip so that when we click ON the marker, the name of the location (aka the city) pops up
   ).add_to(point_map) # and then we add this marker to our blank map

# We should see our map, now with 'markers' or points
point_map

<h3 dir="rtl">کار با فایل‌های شکل</h3>

<div dir="rtl">گاهی اوقات، اگر داده‌ها را به صورت فایل CSV یا GeoJSON داشته باشیم، می‌توانیم از کتابخانه‌های ساده‌تری مانند Folium برای نقشه‌برداری استفاده کنیم. اما به ویژه برای داده‌های غیر نقطه‌ای مانند داده‌های چندضلعی که برای نقشه‌برداری از آنها استفاده می‌کنیم، اغلب داده‌های جغرافیایی را به صورت shapefile دریافت می‌کنیم. فرمت shapefile دارای پسوند '.shp' است.</div>
<div dir="rtl"><a href="https://geopandas.org/en/stable/">Geopandas</a> یک کتابخانه متن‌باز برای نقشه‌برداری است که به جای Folium می‌توانیم از آن برای خواندن shapefile‌ها (و سایر فرمت‌های داده جغرافیایی) استفاده کنیم.</div>


<div dir="rtl">
در اینجا مرزهای استانی <a href="https://geodata.lib.utexas.edu/catalog/stanford-dv126wm3595">ایران</a> در فرمت shapefile از دانشگاه تگزاس، آستین وجود دارد. به عنوان آماده‌سازی، ما این shapefile را قبلاً دانلود کرده و در پوشه "ورودی" ذخیره کرده‌ایم. بیایید این shapefile را در نوت‌بوک بارگیری کنیم.
</div>

In [ ]:
# Importing necessary libraries
import geopandas as gpd

In [ ]:
# Loading in shapefile data with gpd (geopandas)

# the URL to the raw shp file
provinces_shp_raw = "../input/stanford-dv126wm3595-shapefile/dv126wm3595.shp"

# loading in the external shp file using 'read_file'
provinces_shp = gpd.read_file(provinces_shp_raw)

provinces_shp

<div dir="rtl">
اکنون که shapefile به فرمت dataframe است، می‌توانید از یک کتابخانه دیگر (مانند matplotlib) برای دریافت داده‌های داخل dataframe و نمایش آن استفاده کنید.
</div>

In [ ]:
provinces_shp.plot() # plot() is a function of matplotlib

# Remember that the x and y axes are latitude and longitude

<div dir="rtl">
شما حتی می‌توانید یک استان خاص در داخل ایران را جدا کنید تا بررسی کنید. در اینجا، من به طور خاص استان <span style="color: orange">آذربایجان شرقی</span> را با استفاده از ستون <span style="color: orange">name_1</span> که حاوی نام استان‌هاست جدا می‌کنم.
</div>

In [ ]:
provinces_shp[provinces_shp.name_1=="East Azarbaijan"].plot()

<h3 dir="rtl">شما درس مقدمه نقشه نگاری را به پایان رساندید</h3>

<div dir="rtl">ما یاد گرفتیم چگونه نقشه‌ی کروپلت را با استفاده از داده‌های جهانی جمعیت زندانیان و نقشه‌ی نقطه‌ای/نقطه‌ای سفارتخانه‌های ایران در سراسر جهان ایجاد کنیم. همچنین به بررسی کار با فایل‌های شکل، یک فرمت دیگر حاوی اطلاعات جغرافیایی، با استفاده از فایل شکل استان‌های ایران پرداختیم. در زیر، منابعی اضافی برای کمک به شما در نقشه‌برداری با folium، geopandas و همچنین کار با فایل‌های شکل و سایر فرمت‌های داده‌های جغرافیایی آورده شده است!</div>

<h3 dir="rtl">منابع اضافی</h3>

<ul dir="rtl">
  <li>
    نقشه‌برداری نقاط با استفاده از Folium: <a href="https://georgetsilva.github.io/posts/mapping-points-with-folium/">1</a> <a href="https://www.python-graph-gallery.com/312-add-markers-on-folium-map/">2</a> (که جزئیات بیشتری درباره ایجاد نقشه شخصی با نشانگرها برای تجسم داده‌های نقطه‌ای ارائه می‌دهد)
  </li>
  <li>
    استفاده از geopandas برای نمایش فایل‌های shapefile: <a href="https://geopandas.org/en/stable/docs/user_guide/io.html">لینک</a>
  </li>
  <li>
    این آموزش <a href="https://towardsdatascience.com/visualizing-geospatial-data-in-python-e070374fe621">اینجا</a> شامل اطلاعات بیشتری درباره کار با فایل‌های shapefile و تجسم داده‌های جغرافیایی به طور کلی در پایتون است
  </li>
</ul>
